# Getting Started

# LangGraph4j Detailed Explanation

## 1. Basic Concepts of LangGraph

LangGraph is a framework for building complex applications that utilize language models (LLMs). LangGraph4j is the Java implementation, based on the same design philosophy as the Python version of LangChain/LangGraph.

### What is LangGraph4j

- **Definition**: LangGraph4j is a Java library for building complex AI workflows using a graph-based architecture
- **Purpose**: To modularize applications using LLMs and decompose them into manageable components
- **Key Features**: State management, graph-based execution flow, type safety

### Benefits of Graph-Based Design

- **Modularity**: Functions can be implemented as independent nodes, improving reusability
- **Visibility**: Application flow can be visually represented, making complex processes easier to understand
- **Extensibility**: New features can be easily added to existing graphs
- **Debugging**: Execution and state changes of each node are easier to track

## 2. Core Components of LangGraph4j

### StateGraph

StateGraph is the central class of LangGraph4j, defining the graph structure and execution flow.

- **Initialization**: Initialize by specifying state type and schema
- **Compilation**: Validate if the graph is executable and convert it to an execution engine
- **Execution**: Execute with initial state and obtain state changes as a stream

### Nodes and Edges

- **Nodes**: Functions or classes that execute processing (implementing the `NodeAction` interface)
- **Edges**: Define connection relationships and execution order between nodes
- **Special Nodes**: Special nodes `START` and `END` represent the beginning and end of the graph

### State Management

- **AgentState**: Class for maintaining and updating data during graph execution
- **Channel**: Defines state update methods (replacement, addition, merge, etc.)
- **Schema**: Defines state structure and ensures type safety

In [ ]:
var userHomeDir = System.getProperty("user.home");
var localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
var langchain4jVersion = "1.0.1";
var langchain4jbeta = "1.0.1-beta6";
var langgraph4jVersion = "1.6.0-beta2";

Remove packages installed from Jupyter cache

In [ ]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

Add local Maven repository and install required Maven dependencies

In [ ]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
// %dependency /list-repos
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}
%dependency /add net.sourceforge.plantuml:plantuml-mit:1.2024.8
%dependency /resolve

In [ ]:
import net.sourceforge.plantuml.SourceStringReader;
import net.sourceforge.plantuml.FileFormatOption;
import net.sourceforge.plantuml.FileFormat;
import org.bsc.langgraph4j.GraphRepresentation;

void displayDiagram( GraphRepresentation representation ) throws IOException { 
    
    var reader = new SourceStringReader(representation.getContent());

    try(var imageOutStream = new java.io.ByteArrayOutputStream()) {

        var description = reader.outputImage( imageOutStream, 0, new FileFormatOption(FileFormat.PNG));

        var imageInStream = new java.io.ByteArrayInputStream(  imageOutStream.toByteArray() );

        var image = javax.imageio.ImageIO.read( imageInStream );

        display(  image );

    }
}

### 2-1. State Definition

- State: **Holds a list of messages**.

In [ ]:
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.state.Channels;
import org.bsc.langgraph4j.state.Channel;

import java.util.Collections;
import java.util.List;
import java.util.Map;
import java.util.Optional;

// Define the state for our graph
class SimpleState extends AgentState {
    public static final String MESSAGES_KEY = "messages";

    // Define the schema for the state.
    // MESSAGES_KEY will hold a list of strings, and new messages will be appended.
    public static final Map<String, Channel<?>> SCHEMA = Map.of(
            MESSAGES_KEY, Channels.appender(ArrayList::new)
    );

    public SimpleState(Map<String, Object> initData) {
        super(initData);
    }

    public List<String> messages() {
        return this.<List<String>>value("messages")
                .orElse( List.of() );
    }
}

### 2-2. Node Definition

In [ ]:
import org.bsc.langgraph4j.action.NodeAction;
import java.util.Collections;
import java.util.Map;

// Node that adds a greeting
class GreeterNode implements NodeAction<SimpleState> {
    @Override
    public Map<String, Object> apply(SimpleState state) {
        System.out.println("GreeterNode executing. Current messages: " + state.messages());
        return Map.of(SimpleState.MESSAGES_KEY, "Hello from GreeterNode!");
    }
}

// Node that adds a response
class ResponderNode implements NodeAction<SimpleState> {
    @Override
    public Map<String, Object> apply(SimpleState state) {
        System.out.println("ResponderNode executing. Current messages: " + state.messages());
        List<String> currentMessages = state.messages();
        if (currentMessages.contains("Hello from GreeterNode!")) {
            return Map.of(SimpleState.MESSAGES_KEY, "Acknowledged greeting!");
        }
        return Map.of(SimpleState.MESSAGES_KEY, "No greeting found.");
    }
}

### 2-3. Graph Definition and Compilation

In [ ]:
import org.bsc.langgraph4j.StateGraph;
import org.bsc.langgraph4j.GraphStateException;
import static org.bsc.langgraph4j.action.AsyncNodeAction.node_async;
import static org.bsc.langgraph4j.StateGraph.START;
import static org.bsc.langgraph4j.StateGraph.END;

import java.util.List;
import java.util.Map;
import java.io.IOException;

public class SimpleGraphApp {
    
    public static void main(String[] args) throws GraphStateException, IOException {
        // Initialize nodes
        GreeterNode greeterNode = new GreeterNode();
        ResponderNode responderNode = new ResponderNode();

        // Define the graph structure
       var stateGraph = new StateGraph<>(SimpleState.SCHEMA, initData -> new SimpleState(initData))
            .addNode("greeter", node_async(greeterNode))
            .addNode("responder", node_async(responderNode))
            // Define edges
            .addEdge(START, "greeter") // Start with the greeter node
            .addEdge("greeter", "responder")
            .addEdge("responder", END)   // End after the responder node
            ;
        // Compile the graph
        var compiledGraph = stateGraph.compile();

        // Run the graph
        // The `stream` method returns an AsyncGenerator.
        // For simplicity, we'll collect results. In a real app, you might process them as they arrive.
        // Here, the final state after execution is the item of interest.
        
        for (var item : compiledGraph.stream( Map.of( SimpleState.MESSAGES_KEY, "Let's, begin!" ) ) ) {
            System.out.println( item );
        }

        var representation = stateGraph.getGraph( GraphRepresentation.Type.PLANTUML, "Graph", false );
        try {
            // Display the diagram
            displayDiagram(representation);
        } catch (IOException e) {
            System.err.println("Error displaying diagram: " + e.getMessage());
        }
    }
}



In [ ]:
SimpleGraphApp.main(new String[]{});